In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime as daee
from itertools import combinations
import matplotlib.pyplot as plt
import json
import requests
import statsmodels.api as sm
import scipy.optimize as spop

In [23]:

pricehistpastday=pd.read_csv("price_histpastday1.csv")
pricehistpastday2=pd.read_csv("price_histpastday2.csv")
pricehistpastweek=pd.read_csv("price_histpastweek.csv")
pricehistpastweek2=pd.read_csv("price_histpastweek2.csv")

pricelist=[pricehistpastday,pricehistpastday2,pricehistpastweek,pricehistpastweek2]
ponderation=[0.6,0.2,0.15,0.05]
x=0
dimcorrel=pricehistpastday.corr(numeric_only=True)
for col in dimcorrel.columns:
    dimcorrel[col].values[:] = 0
for j in pricelist:
    correlationmatrix=j.corr()
    correlationmatrix=correlationmatrix.mul(ponderation[x])
    dimcorrel=dimcorrel.add(correlationmatrix,fill_value=0)
   
        

    x+=1
    

C:\Users\jeanc\AppData\Local\Temp\ipykernel_27324\643795661.py:13: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlationmatrix=j.corr()


In [3]:
dimcorrel
listdic={}
listlist=[]
for column in dimcorrel.columns[1:]:
    dimdiss=dimcorrel[column].nlargest(2)
    listdic[dimdiss.index[0]]=[dimdiss.index[1],dimdiss[1]]
    listlist.append([dimdiss.index[0],dimdiss.index[1],dimdiss[1]])

    


In [19]:
import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.optimize as spop
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime as daee
from itertools import combinations
import json
import requests
import statsmodels.api as sm
def get_histweek(ticker,period,interval,week):
    
    today = datetime.datetime.today()
    last_week_start = today - datetime.timedelta(days=today.weekday(), weeks=week)
    
    last_week_start=last_week_start.replace(hour=9,minute=29)
    last_week_end = last_week_start + datetime.timedelta(days=365)
    price_history=yf.Ticker(ticker).history(start=last_week_start,end=last_week_end,interval=interval)["Close"]
   
    return(price_history.dropna())

class CointegrationA:
    
    def datasetprovider(stock1,stock2,week,period,backtrack):
        stocks=[stock1,stock2]
        data=pd.DataFrame()
        returns=pd.DataFrame()
        for stock in stocks:
            prices=get_histweek(stock,week,period,backtrack)
        
            data[stock]=prices
            returns[stock]=np.append(data[stock][1:].reset_index(drop=True)/data[stock][:-1].reset_index(drop=True)-1,0)
        return(returns,data)

    def cointegrationCacl(stock1,stock2,data,returns,fee,t_threshold,window):
        gross_returns=np.array([])
        net_returns=np.array([])
        t_s=np.array([])
        
        x=0
        tradelog={}
        for t in range(window,len(data)):
            x+=1

            
            def unit_root(b):
                a=np.average(data[stock2][t-window:t]-b*data[stock1][t-window:t])
                fair_value=a+b*data[stock1][t-window:t]
                diff=np.array(fair_value-data[stock2][t-window:t])
                diff_diff=diff[1:]-diff[:-1]
                
                reg=sm.OLS(diff_diff, diff[:-1])
                res=reg.fit()
                
                return (res.params[0]/res.bse[0])
            res1=spop.minimize(unit_root,data[stock2][t]/data[stock1][t],method='Nelder-Mead')
            t_opt=res1.fun
            b_opt=float(res1.x)
            a_opt=np.average(data[stock2][t-window:t]-b_opt*data[stock1][t-window:t])
            fair_value=a_opt+b_opt*data[stock1][t]

            if t==window:
                old_signal=0
            if t_opt > t_threshold:
                signal=0
                gross_return=0
            else:
                signal=np.sign(fair_value-data[stock2][t])
                gross_return=signal*returns[stock2][t]-signal*returns[stock1][t]
            fees=fee*abs(signal-old_signal)
            net_return=gross_return-fees
            gross_returns=np.append(gross_returns,gross_return)
            net_returns=np.append(net_returns,net_return)
            t_s=np.append(t_s,t_opt)

            
          
            
            tradeloglist=["no trading","long position on "+ stock2+" and short position on "+stock1,"long position on "+ stock1 +" and short position on "+stock2]
            if signal==0:
                listindex=0
            elif signal==1:
                listindex=1
            else:
                listindex=2
            tradingpair=stock1+"-"+stock2+ " on "+str(data.index[t])
            
            
            tradelog[tradingpair]=[tradeloglist[listindex],"gross period return "+str(round(gross_return*100,2))+" %","net period return "+str(round(net_return*100,2))+" %"]
            old_signal=signal
        return(str(round(np.prod(1+net_returns)*100-100,2))+"%",tradelog)

stockchoice=['DHI','PHM']

dataset=CointegrationA.datasetprovider(stock1=stockchoice[0],stock2=stockchoice[1],week="5w",period="30m",backtrack=4)

everything=CointegrationA.cointegrationCacl(stockchoice[0],stockchoice[1],dataset[1],dataset[0],0.0001,-3.5,35)
    
    





In [20]:

print(everything[0])
everything[1]



1.42%


{'DHI-PHM on 2023-08-02 14:00:00-04:00': ['no trading',
  'gross period return 0 %',
  'net period return 0.0 %'],
 'DHI-PHM on 2023-08-02 14:30:00-04:00': ['long position on DHI and short position on PHM',
  'gross period return 0.15 %',
  'net period return 0.14 %'],
 'DHI-PHM on 2023-08-02 15:00:00-04:00': ['no trading',
  'gross period return 0 %',
  'net period return -0.01 %'],
 'DHI-PHM on 2023-08-02 15:30:00-04:00': ['no trading',
  'gross period return 0 %',
  'net period return 0.0 %'],
 'DHI-PHM on 2023-08-03 09:30:00-04:00': ['no trading',
  'gross period return 0 %',
  'net period return 0.0 %'],
 'DHI-PHM on 2023-08-03 10:00:00-04:00': ['no trading',
  'gross period return 0 %',
  'net period return 0.0 %'],
 'DHI-PHM on 2023-08-03 10:30:00-04:00': ['no trading',
  'gross period return 0 %',
  'net period return 0.0 %'],
 'DHI-PHM on 2023-08-03 11:00:00-04:00': ['no trading',
  'gross period return 0 %',
  'net period return 0.0 %'],
 'DHI-PHM on 2023-08-03 11:30:00-04:00

In [ ]:
#y=ax+b+u->(x1,x2,x3,x4)/